In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)   

In [18]:
# Synthetic Cohort Data
order_timestamp = ['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
                                 '2020-02-01', '2020-03-01', '2020-04-01',
                                               '2020-03-01', '2020-04-01']
cust_id = [1, 1, 1, 1, 2, 2, 2, 3, 3]
amount_spent = [100 + (10*x)*i for i, x in enumerate(cust_id)]

df = pd.DataFrame(
    list(zip(order_timestamp, cust_id, amount_spent)),
    columns=["timestamp", "cust_id", "amount_spent"]
)
df["timestamp"] = pd.to_datetime(df["timestamp"])

print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
timestamp       9 non-null datetime64[ns]
cust_id         9 non-null int64
amount_spent    9 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 344.0 bytes
None
   timestamp  cust_id  amount_spent
0 2020-01-01        1           100
1 2020-02-01        1           110
2 2020-03-01        1           120
3 2020-04-01        1           130
4 2020-02-01        2           180
5 2020-03-01        2           200
6 2020-04-01        2           220
7 2020-03-01        3           310
8 2020-04-01        3           340


In [19]:
# Assign cohort membership
first_purchase_dates = (df
    .groupby("cust_id")['timestamp']
    .min()
    .reset_index()
    .rename(columns={'timestamp': 'acq_timestamp'}))
first_purchase_dates

,cust_id,acq_timestamp
0,1,2020-01-01
1,2,2020-02-01
2,3,2020-03-01


In [20]:
df = (df
      .set_index('cust_id')
      .join(first_purchase_dates.set_index('cust_id')))

df

,timestamp,amount_spent,acq_timestamp
cust_id,,,
1,2020-01-01,100,2020-01-01
1,2020-02-01,110,2020-01-01
1,2020-03-01,120,2020-01-01
1,2020-04-01,130,2020-01-01
2,2020-02-01,180,2020-02-01
2,2020-03-01,200,2020-02-01
2,2020-04-01,220,2020-02-01
3,2020-03-01,310,2020-03-01
3,2020-04-01,340,2020-03-01


In [21]:
acq_years = df["acq_timestamp"].dt.year.apply(str)
acq_months = df["acq_timestamp"].dt.month.apply(str)
acq_period = acq_years.str.cat(acq_months, sep="-")
df["acq_period"] = acq_period
df

,timestamp,amount_spent,acq_timestamp,acq_period
cust_id,,,,
1,2020-01-01,100,2020-01-01,2020-1
1,2020-02-01,110,2020-01-01,2020-1
1,2020-03-01,120,2020-01-01,2020-1
1,2020-04-01,130,2020-01-01,2020-1
2,2020-02-01,180,2020-02-01,2020-2
2,2020-03-01,200,2020-02-01,2020-2
2,2020-04-01,220,2020-02-01,2020-2
3,2020-03-01,310,2020-03-01,2020-3
3,2020-04-01,340,2020-03-01,2020-3


In [22]:
# Compute Age
df['age'] = (df["timestamp"].dt.year - df["acq_timestamp"].dt.year) * 12 + (df["timestamp"].dt.month - df["acq_timestamp"].dt.month)
df

,timestamp,amount_spent,acq_timestamp,acq_period,age
cust_id,,,,,
1,2020-01-01,100,2020-01-01,2020-1,0
1,2020-02-01,110,2020-01-01,2020-1,1
1,2020-03-01,120,2020-01-01,2020-1,2
1,2020-04-01,130,2020-01-01,2020-1,3
2,2020-02-01,180,2020-02-01,2020-2,0
2,2020-03-01,200,2020-02-01,2020-2,1
2,2020-04-01,220,2020-02-01,2020-2,2
3,2020-03-01,310,2020-03-01,2020-3,0
3,2020-04-01,340,2020-03-01,2020-3,1


In [23]:
cohort_trans_log = (df
    .groupby(["acq_timestamp", "timestamp"])["amount_spent"]
    .sum()
    .reset_index()
    .rename(columns={"acq_timestamp":"cohort"}))
cohort_trans_log

,cohort,timestamp,amount_spent
0,2020-01-01,2020-01-01,100
1,2020-01-01,2020-02-01,110
2,2020-01-01,2020-03-01,120
3,2020-01-01,2020-04-01,130
4,2020-02-01,2020-02-01,180
5,2020-02-01,2020-03-01,200
6,2020-02-01,2020-04-01,220
7,2020-03-01,2020-03-01,310
8,2020-03-01,2020-04-01,340


In [24]:
def get_date(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

def compute_cohort_ages(trans_log, acq_timestamp, order_timestamp, by):
    if by == "month":
        order_years, order_months, _ = get_date(trans_log, order_timestamp)
        acq_years, acq_months, _ = get_date(trans_log, acq_timestamp)
        trans_log["age"] = (order_years - acq_years) * 12 + (order_months - acq_months)
    return trans_log

cohort_trans_log = compute_cohort_ages(
    cohort_trans_log,
    acq_timestamp="cohort",
    order_timestamp="timestamp",
    by="month"
)

cohort_trans_log

,cohort,timestamp,amount_spent,age
0,2020-01-01,2020-01-01,100,0
1,2020-01-01,2020-02-01,110,1
2,2020-01-01,2020-03-01,120,2
3,2020-01-01,2020-04-01,130,3
4,2020-02-01,2020-02-01,180,0
5,2020-02-01,2020-03-01,200,1
6,2020-02-01,2020-04-01,220,2
7,2020-03-01,2020-03-01,310,0
8,2020-03-01,2020-04-01,340,1


## Visualizations

In [25]:
cohort_trans_log.pivot_table(
    index = "cohort",
    columns = "timestamp",
    values= "amount_spent" 
)

timestamp,2020-01-01,2020-02-01,2020-03-01,2020-04-01
cohort,,,,
2020-01-01,100.0,110.0,120.0,130.0
2020-02-01,NaN,180.0,200.0,220.0
2020-03-01,NaN,NaN,310.0,340.0


In [26]:
px.line(cohort_trans_log,
        x="timestamp",
        y="amount_spent",
        color="cohort")

In [27]:
px.line(cohort_trans_log,
        x="age",
        y="amount_spent",
        color="cohort")

In [54]:
print(cohort_trans_log)
import plotly.graph_objects as go


plt_layout = go.Layout(
    title = "Customer Cohort Chart",
    xaxis_title="Period",
    yaxis_title="Revenue",
    xaxis = {'showgrid': False,
             'zeroline': True,
             'linecolor': 'rgb(100, 100, 100)',
             'ticks': 'outside'},
    yaxis = {'showgrid': False,
             'zeroline': True,
             'linecolor': 'rgb(100, 100, 100)',
             'ticks': 'outside'
            },
    plot_bgcolor = 'rgba(0, 0, 0, 0)'
)
plt = go.Figure(
    layout = plt_layout
)
plt.add_trace(go.Scatter(
    name="cohort_1",
    x=cohort_trans_log[cohort_trans_log["cohort"] == "2020-01-01"]["timestamp"],
    y=cohort_trans_log[cohort_trans_log["cohort"] == "2020-01-01"]["amount_spent"],
    stackgroup='one',
    mode='none'
))
plt.add_trace(go.Scatter(
    name="cohort_2",
    x=cohort_trans_log[cohort_trans_log["cohort"] == "2020-02-01"]["timestamp"],
    y=cohort_trans_log[cohort_trans_log["cohort"] == "2020-02-01"]["amount_spent"],
    stackgroup='one',
    mode='none'
))
plt.add_trace(go.Scatter(
    name="cohort_3",
    x=cohort_trans_log[cohort_trans_log["cohort"] == "2020-03-01"]["timestamp"],
    y=cohort_trans_log[cohort_trans_log["cohort"] == "2020-03-01"]["amount_spent"],
    stackgroup='one',
    mode='none'
))
plt.show()


      cohort  timestamp  amount_spent  age
0 2020-01-01 2020-01-01           100    0
1 2020-01-01 2020-02-01           110    1
2 2020-01-01 2020-03-01           120    2
3 2020-01-01 2020-04-01           130    3
4 2020-02-01 2020-02-01           180    0
5 2020-02-01 2020-03-01           200    1
6 2020-02-01 2020-04-01           220    2
7 2020-03-01 2020-03-01           310    0
8 2020-03-01 2020-04-01           340    1
